In [17]:
# 3. In 2. above you will notice that some entries have only the code and the name is missing. 
# Create a dataframe with the missing names filled in.

import pandas as pd
import numpy as np
import json
import collections
from pandas.io.json import json_normalize

pd.options.display.max_columns = 999
pd.options.display.max_rows = 500

wbproj = pd.read_json('/Users/charlesgreen/desktop/data/World Bank Projects/world_bank_projects.json')

#Viewing df information
#wbproj.info()

In [5]:
#mjtheme has 9 NaN values.  View lines with NaN. 
#Note:  mjthemecode has no NaN values.
wbproj[wbproj['mjtheme'].isnull()]

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,docty,envassesmentcategorycode,grantamt,ibrdcommamt,id,idacommamt,impagency,lendinginstr,lendinginstrtype,lendprojectcost,majorsector_percent,mjsector_namecode,mjtheme,mjtheme_namecode,mjthemecode,prodline,prodlinetext,productlinetype,project_abstract,project_name,projectdocs,projectfinancialtype,projectstatusdisplay,regionname,sector,sector1,sector2,sector3,sector4,sector_namecode,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
12,{'$oid': '52b213b38594d8a2be17c78c'},2014,October,2013-10-24T00:00:00Z,GOVERNMENT OF GHANA,2019-06-30T00:00:00Z,Republic of Ghana!$!GH,GH,Republic of Ghana,Ghana,"Project Appraisal Document,Integrated Safeguar...",C,0,0,P144140,97000000,MINISTRY OF COMMUNICATIONS,Specific Investment Loan,IN,97000000,"[{'Percent': 100, 'Name': 'Information and com...","[{'code': 'CX', 'name': 'Information and commu...",NaN,"[{'code': '4', 'name': ''}]",4,PE,IBRD/IDA,L,{'cdata': 'The development objective of the e-...,GH eTransform Ghana,"[{'DocDate': '26-SEP-2013', 'EntityID': '00045...",IDA,Active,Africa,[{'Name': 'General information and communicati...,"{'Percent': 100, 'Name': 'General information ...",NaN,NaN,NaN,"[{'code': 'CZ', 'name': 'General information a...",CZ,IBRD,Active,N,"{'Percent': 0, 'Name': ''}",NaN,NaN,97000000,97000000,http://www.worldbank.org/projects/P144140/gh-e...
67,{'$oid': '52b213b38594d8a2be17c7c3'},2014,August,2013-08-01T00:00:00Z,OFFICE NATIONAL DE LA STATISTIQUE,2015-06-30T00:00:00Z,Islamic Republic of Mauritania!$!MR,MR,Islamic Republic of Mauritania,Mauritania,NaN,C,450000,0,P131777,0,ONS,Technical Assistance Loan,IN,450000,"[{'Percent': 100, 'Name': 'Public Administrati...","[{'code': 'BX', 'name': 'Public Administration...",NaN,"[{'code': '7', 'name': ''}]",7,RE,Recipient Executed Activities,L,NaN,Mauritania - TFSCB - Support to NSDS implement...,NaN,OTHER,Active,Africa,[{'Name': 'Central government administration'}],"{'Percent': 100, 'Name': 'Central government a...",NaN,NaN,NaN,"[{'code': 'BC', 'name': 'Central government ad...",BC,IBRD,Active,N,"{'Percent': 0, 'Name': ''}",NaN,NaN,0,450000,http://www.worldbank.org/projects/P131777?lang=en
87,{'$oid': '52b213b38594d8a2be17c7d7'},2013,July,2013-07-11T00:00:00Z,MINISTRY OF FINANCE,2018-03-31T00:00:00Z,Kingdom of Morocco!$!MA,MA,Kingdom of Morocco,Morocco,"Procurement Plan,Project Information Document,...",C,5500000,0,P144134,0,MINISTRY OF YOUTH AND SPORTS,Technical Assistance Loan,IN,5500000,"[{'Percent': 100, 'Name': 'Health and other so...","[{'code': 'JX', 'name': 'Health and other soci...",NaN,"[{'code': '11', 'name': ''}]",11,RE,Recipient Executed Activities,L,NaN,Strengthening micro-entrepreneurship for disad...,"[{'DocDate': '20-AUG-2013', 'EntityID': '00044...",OTHER,Active,Middle East and North Africa,[{'Name': 'Other social services'}],"{'Percent': 100, 'Name': 'Other social services'}",NaN,NaN,NaN,"[{'code': 'JB', 'name': 'Other social services'}]",JB,IBRD,Active,N,"{'Percent': 0, 'Name': ''}",NaN,NaN,0,5500000,http://www.worldbank.org/projects/P144134/stre...
102,{'$oid': '52b213b38594d8a2be17c7e6'},2013,June,2013-06-27T00:00:00Z,MINISTRY OF FINANCE,2013-12-31T00:00:00Z,Nepal!$!NP,NP,Nepal,Nepal,"Implementation Status and Results Report,Progr...",NaN,0,0,P129929,30000000,MINISTRY OF FINANCE,Development Policy Lending,AD,30000000,"[{'Percent': 78, 'Name': 'Finance'}, {'Percent...","[{'code': 'FX', 'name': 'Finance'}, {'code': '...",NaN,"[{'code': '2', 'name': ''}]",2,PE,IBRD/IDA,L,{'cdata': 'The objectives of the Financial Sec...,NP: Financial sector stability DPC,"[{'DocDate': '23-OCT-2013', 'EntityID': '09022...",IDA,Closed,South Asia,"[{'Name': 'Banking'}, {'Name': 'General financ...","{'Percent': 78, 'Name': 'Banking'}","{'Percent': 22, 'Name': 'General finance sector'}",NaN,NaN,"[{'code': 'FA', 'name': 'Banking'}, {'code': '...","

In [18]:

#Viewing df columns, unstack nested 'mjtheme_namecode' column
theme_codes = json_normalize(wbproj.to_dict('list'), ['mjtheme_namecode']).unstack().apply(pd.Series)

#theme_codes

In [22]:
#Check count of individual codes
theme_codes['code'].value_counts()


11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
3      15
Name: code, dtype: int64

In [19]:
#Create dictionary to allow to identify theme codes by name.
#Begin with boolean argument to remove NAN values and empty spaces.
nix = (theme_codes['name'] == theme_codes['name']) & (theme_codes['name'] != "")

#theme_codes.loc[~nix]

In [14]:
#Create dictionary cdnms
cdnms = theme_codes.loc[nix].groupby(by='code').first().to_dict()['name']

cdnms

{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

In [20]:
#Create new/Copy dataframe from original.
wbproj2 = wbproj

#wbproj2

In [9]:
#Beginning to create loop to replace NaN's. Find null values in mjtheme column. 
for index, row in enumerate(wbproj2['mjtheme'].isnull()):
    if row == True:

#Printing index to verify 9 null value locations.
        print(index)

12
67
87
102
103
121
169
276
469


In [15]:
#Earlier note for mjthemecode:  Given 0 null values, code allows reference to dictionary.
for index, row in enumerate(wbproj2['mjtheme'].isnull()):
    if row == True:
        
#Find appropriate mjthemecode.
        x = wbproj2.loc[index,'mjthemecode']
    
#Reference code names dictionary for appropriate code name    
        for key, value in cdnms.items():
            if x == key:
        
#Replace null value with appropriate mjtheme name.        
                wbproj2.loc[index,'mjtheme'] = value

In [21]:
#verify mjtheme now has 0 null values.
#wbproj2.info()